In [5]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

In [4]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LoadingData").getOrCreate()

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://spot-i-fy-bucket.s3.us-east-2.amazonaws.com/tracks.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("tracks.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+--------------------+--------------------+----------+-----------+--------+-------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|                  id|                name|popularity|duration_ms|explicit|            artists|          id_artists|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|
+--------------------+--------------------+----------+-----------+--------+-------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|35iwgR4jXetI318WE...|               Carve|         6|     126903|       0|            ['Uli']|['45tIt06XoI0Iio4...|  1922-02-22|       0.645| 0.445|  0| -13.338|   1|      0.451|       0.674|           0.744|   0.151|  0.127|104.851|             3|


In [6]:
# Print our schema
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- explicit: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- id_artists: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- time_signature: string (nullable = true)



In [7]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://spot-i-fy-bucket.s3.us-east-2.amazonaws.com/artists.csv"
spark.sparkContext.addFile(url)
a_df = spark.read.csv(SparkFiles.get("artists.csv"), sep=",", header=True)

# Show DataFrame
a_df.show()

+--------------------+---------+------+--------------------+----------+
|                  id|followers|genres|                name|popularity|
+--------------------+---------+------+--------------------+----------+
|0DheY5irMjBUeLybb...|      0.0|    []|Armid & Amir Zare...|         0|
|0DlhY15l3wsrnlfGi...|      5.0|    []|         ปูนา ภาวิณี|         0|
|0DmRESX2JknGPQyO1...|      0.0|    []|               Sadaa|         0|
|0DmhnbHjm1qw6NCYP...|      0.0|    []|           Tra'gruda|         0|
|0Dn11fWM7vHQ3rinv...|      2.0|    []|Ioannis Panoutsop...|         0|
|0DotfDlYMGqkbzfBh...|      7.0|    []|       Astral Affect|         0|
|0DqP3bOCiC48L8SM9...|      1.0|    []|           Yung Seed|         0|
|0Drs3maQb99iRglyT...|      0.0|    []|               Wi'Ma|         0|
|0DsPeAi1gxPPnYjgp...|      0.0|    []|             lentboy|         0|
|0DtvnTxgZ9K5YaPS5...|     20.0|    []|            addworks|         0|
|0Du4t8UK1lBD8TnOI...|      1.0|    []|         Harrbazzter|    

In [8]:
# Import dependencies for Machine Learning
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import fcluster, linkage, dendrogram
import pandas as pd
import numpy as np
from pathlib import Path
#import hvplot.pandas
#from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn.metrics import confusion_matrix

In [9]:
# Convert Spark DataFrame to Pandas DataFrame
artists_df = a_df.toPandas()
a_df.head()

Row(id='0DheY5irMjBUeLybbCUEZ2', followers='0.0', genres='[]', name='Armid & Amir Zare Pashai feat. Sara Rouzbehani', popularity='0')

In [10]:
# Convert Spark DataFrame to Pandas DataFrame
track_df = df.toPandas()
track_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.445,0,-13.338,1,0.451,0.674,0.744,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.263,0,-22.136,1,0.957,0.797,0.0,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.177,1,-21.18,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918,0.104,0.397,169.98,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.158,3,-16.9,0,0.039,0.989,0.13,0.311,0.196,103.22,4


In [11]:
artists_df['artists'] = artists_df['name']
artists_df.head()

,id,followers,genres,name,popularity,artists
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0,Armid & Amir Zare Pashai feat. Sara Rouzbehani
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0,ปูนา ภาวิณี
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0,Sadaa
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,[],Tra'gruda,0,Tra'gruda
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,[],Ioannis Panoutsopoulos,0,Ioannis Panoutsopoulos


In [12]:
# Merge the two dataframes
# How to merge when columns have different name
music_df = track_df.merge(artists_df, how='left')
music_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,followers,genres
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.445,0,-13.338,1,0.451,0.674,0.744,0.151,0.127,104.851,3,NaN,NaN
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.263,0,-22.136,1,0.957,0.797,0.0,0.148,0.655,102.009,1,NaN,NaN
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.177,1,-21.18,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5,NaN,NaN
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918,0.104,0.397,169.98,3,NaN,NaN
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.158,3,-16.9,0,0.039,0.989,0.13,0.311,0.196,103.22,4,NaN,NaN


In [13]:
# Exploratory Data to understand how we need to preprocess
# Explore popularity range - over 75? over 50?
music_df.dtypes

id                  object
name                object
popularity          object
duration_ms         object
explicit            object
artists             object
id_artists          object
release_date        object
danceability        object
energy              object
key                 object
loudness            object
mode                object
speechiness         object
acousticness        object
instrumentalness    object
liveness            object
valence             object
tempo               object
time_signature      object
followers           object
genres              object
dtype: object

In [14]:
# Preprocessing
# Drop columns that won't be useful to clustering that might predict genre
spotify_df = music_df.drop(columns=['id', 'name', 'duration_ms', 'explicit', 'artists', 'release_date', 'followers', 'genres', 'id_artists'])
spotify_df.head()

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,6,0.645,0.445,0,-13.338,1,0.451,0.674,0.744,0.151,0.127,104.851,3
1,0,0.695,0.263,0,-22.136,1,0.957,0.797,0.0,0.148,0.655,102.009,1
2,0,0.434,0.177,1,-21.18,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,0,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918,0.104,0.397,169.98,3
4,0,0.402,0.158,3,-16.9,0,0.039,0.989,0.13,0.311,0.196,103.22,4


In [15]:
# Convert to numeric data
# df["A"] = pd.to_numeric(df["A"], downcast="float")
cols = ['popularity', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'key', 'mode', 'time_signature']
spotify_df[cols] = spotify_df[cols].apply(pd.to_numeric, errors='coerce', axis=1)

In [16]:
# Filter table by tracks with popularity above 50
spotify_df = spotify_df[spotify_df['popularity'] >= 50]
spotify_df.head()

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
1857,51.0,0.515,0.2220,0.0,-16.918,0.0,0.0575,0.821,0.001930,0.190,0.350,98.358,4.0
3857,51.0,0.626,0.7440,2.0,-9.189,0.0,0.0662,0.847,0.892000,0.145,0.259,113.117,4.0
4857,55.0,0.673,0.3770,0.0,-14.141,1.0,0.0697,0.586,0.000000,0.332,0.713,88.973,4.0
6505,50.0,0.482,0.2870,1.0,-9.108,0.0,0.0482,0.981,0.000017,0.300,0.536,69.804,4.0
8095,65.0,0.504,0.0644,2.0,-14.754,0.0,0.0408,0.972,0.000004,0.174,0.403,106.994,5.0


In [ ]:
# Feature selection: 
# drop popularity? Maybe not because some genres are more popular than others
#spotify_df_filter = spotify_df.drop(columns='popularity')

In [17]:
# Drop null values
spotify_df = spotify_df.dropna()
spotify_df.head()

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
1857,51.0,0.515,0.2220,0.0,-16.918,0.0,0.0575,0.821,0.001930,0.190,0.350,98.358,4.0
3857,51.0,0.626,0.7440,2.0,-9.189,0.0,0.0662,0.847,0.892000,0.145,0.259,113.117,4.0
4857,55.0,0.673,0.3770,0.0,-14.141,1.0,0.0697,0.586,0.000000,0.332,0.713,88.973,4.0
6505,50.0,0.482,0.2870,1.0,-9.108,0.0,0.0482,0.981,0.000017,0.300,0.536,69.804,4.0
8095,65.0,0.504,0.0644,2.0,-14.754,0.0,0.0408,0.972,0.000004,0.174,0.403,106.994,5.0


In [4]:
# Use Hierarchical clustering
# Normalize data
from sklearn.preprocessing import normalize
data_scaled = normalize(spotify_df)
data_scaled = pd.DataFrame(data_scaled, columns=spotify_df.columns)
data_scaled.head()

NameError: ignored

In [2]:
import plotly.figure_factory as ff

In [3]:
# Create the dendrogram
fig = ff.create_dendrogram(data_scaled, color_threshold=0)
fig.update_layout(width=800, height=500)
fig.show()

NameError: ignored

In [23]:
# Create hierarchal clustering array using n_clusters determined from dendrogram analysis
from sklearn.cluster import AgglomerativeClustering
cluster = AgglomerativeClustering(n_clusters=10, affinity='euclidean', linkage='ward')  
cluster.fit_predict(data_scaled)

NameError: ignored

In [ ]:
# Plot scatter plot of clusters

In [ ]:
# Does using PCA help? Some clusters are opposite/inverse of others, so it might hurt

In [ ]:
# Go through a few models. 3 maybe? Why do I think the best performing model is best performing?

# Notes from class! Is this data set actually labeled? Maybe I want to see if this can predict genre?
# How would I design a neural network? Use that loop to pick the number of features?
# How accurate/good is each model?
# Were we able to use a simple model?
# Do we get similar results each time? 

In [ ]:
# I'll do k means clustering and round it out with a neural network
# Question is, can we predict the genre lable?
# Talk about stuff over time for the tableau bit

In [ ]:
#K Means Clustering
#Begin with elbow graph to determine how many clusters we might want
inertia = []
k = list(range(1, 11))
# Calculate the inertia for the range of K values
for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(df_shopping)
   inertia.append(km.inertia_)
#Create elbow curve in hvPlot
elbow_data = {"k":k, "inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x-"k", y="inertia", xticks=k, title="Elbow Curve")

In [ ]:
# Set X 
X = spotify_df
X_scaled = StandardScaler().fit_transform(X)

In [ ]:
# Create K Means Model with X clusters (from elbow plot)
model = KMeans(n_clusters=3, random_state=42).fit(X_scaled)

In [ ]:
# Calculate predicted values.
y_pred = model.predict(X_scaled)

In [ ]:
# Add predicted values onto the original dataframe
df_y = pd.DataFrame(y_pred, columns=['Cluster'])
combined = spotify_df.join(df_y, how='inner')
combined.head()

In [ ]:
# Plot clusters
# Create a scatterplot of spotify_df
#df_iris.hvplot.scatter(x="SOME COLUMN", y="SOME COLUMN", by="SOME COLUMN")
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [ ]:
#Neural Network
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf

In [ ]:
# Use the preprocessed data from the previous 
# Wait can I even use a normal neural network? 
# yes if I'm trying to guess popularity - simple under 50/over 50 or log regression?